# 문제 : 문자열 매칭 알고리즘 구현

문자열 집합 S가 존재한다 ex) S = {"www","xman","yman"}. 임의의 문자열이 N개 주어졌을때 각 임의의 문자열 내에 집합 S에 속하는 문자열이 존재하는지 판별하시오.

아래의 문제를 순서대로 작성하세요. 1번만 푸셔도 됩니다.

1. 문자열의 여러 부분 문자열 중 하나라도 집합 S에 있으면 'YES'를 출력하고, 아무것도
없으면 'NO'를 출력한다.
2. 주어진 문자열의 여러부분 문자열 중 처음 매칭된 패턴과 문자열 시작 포지션을 출력한
다.
3. 주어진 문자열의 여러부분 문자열 중 매칭 가능한 모든 패턴과 문자열 시작 포지션을 출
력한다.

#입력

첫째 줄에 집합 S의 크기 N이 주어진다. (1 ≤ N ≤ 1000)
다음 N개 줄에 집합 S의 원소들이 주어진다. 이 문자열의 길이는 100을 넘지 않는다.

다음 줄에 답을 판별해야 하는 문자열의 개수 Q가 주어진다. (1 ≤ Q ≤ 1000)

다음 Q개 줄에 답을 판별해야 하는 문자열이 주어진다. 이 문자열의 길이는 10000을 넘지 않는다.

입력으로 주어지는 모든 문자열은 알파벳 소문자로만 이루어져 있다.

#출력

Q개 줄에 각 문자열에 대한 답을 출력한다.

 #예제 입력

9(문자열 집합S의 원소 개수) 
"aaa",""aaaabb", ""aabbcc", "abb", "bcc", "bbcc", "aabbccdd", "aaabb”, “cccd” (comma, new line 구분은 자유) 

1(매칭 대상 문자열 수) 
"aaaabbaabbccdd" 


출력형식은 무관합니다.
1. 문자열 집합 중 “cccd”만 주어진 문자열의 부분 문자열에 속하지 않으므로 YES, YES, YES, YES, YES, YES, YES, YES, NO 이다.

2. 아래와같이 첫번째로 매칭된 패턴의 시작포지션과 문자열 패턴을 출력한다.

pos = 0, pattern = aaa

pos = 0, pattern = aaaabb

pos = 6, pattern = aabbcc

pos = 7, pattern = abb

pos = 9, pattern = bcc

pos = 8, pattern = bbcc

pos = 6, pattern = aabbccdd

pos = 1, pattern = aaabb

3. 아래와같이 가능한 모든 문자열 패턴의 시작 포지션과 문자열 패턴을 출력한다.

pos = 0, pattern = aaa

pos = 1, pattern = aaa

pos = 1, pattern = aaabb

pos = 3, pattern = abb

pos = 0, pattern = aaaabb

pos = 7, pattern = abb

pos = 8, pattern = bbcc

pos = 9, pattern = bcc

pos = 6, pattern = aabbcc

pos = 6, pattern = aabbccdd

In [132]:
from collections import deque

class State:
    sid = None
    value = None
    isFinal = False
    tranList = None
    failState = 0
    outputSet = None

    def __init__(self, sid, val):
        self.sid = sid
        self.value = val
        self.tranList = {}
        self.failState = 0
        self.outputSet = set()

    def goto(self, key):
        if key in self.tranList:
            return self.tranList[key]

    def addOutput(self, key):
        self.outputSet = self.outputSet ^ key

    def display(self):
        print("State:", self.sid, "Outgoing:", len(self.tranList.keys()), "Failure:", self.failState)

        if self.isFinal == True:
            print(" +--Output:", self.outputSet)

        for node in self.tranList.keys():
            s = self.tranList[node]
            s.display()
class main:
    root = None
    sid = 0
    table = {}

    def __init__(self):
        self.root = State(0, None)
        self.table[0] = self.root

    def addKeyword(self, keyword):
        current = self.root

        for key in keyword:
            if key not in current.tranList:
                self.sid = self.sid + 1
                current.tranList[key] = State(self.sid, key)
                self.table[self.sid] = current.tranList[key]

            current = current.tranList[key]

        current.isFinal = True
        current.outputSet.add(keyword)
    def setFailure(self):
        queue = deque()
        current = self.root

        for k in self.root.tranList:
            queue.append(self.root.tranList[k])

        while len(queue) != 0:
            r = queue.popleft()
            for k in r.tranList:
                queue.append(r.tranList[k])
                nd = r.tranList[k]
                sid = r.failState
                value = nd.value
                current = self.table[sid]

                while True:
                    if current.goto(value) == None and current.sid != 0:
                        new_sid = current.failState
                        current = self.table[new_sid]
                    else:
                        break
                child = current.goto(value)

                if child == None:
                    nd.failState = current.sid
                else:
                    nd.failState = child.sid

                nd.addOutput(self.table[nd.failState].outputSet)

    def findString(self, str):
        current = self.root

        count = 0
        for key in str:
            # print("Check:", key)
            while True:
                if current.goto(key) == None and current.sid != 0:
                    current = self.table[current.failState]
                    # print("failure:", current.sid)
                else:
                    child = current.goto(key)
                    break
            if child != None:
                current = child
                if len(child.outputSet) > 0:
                    return True

        return False

    def display(self):
        self.root.display()

In [135]:
x = main()
pattern =  ['aaa','aaaabb', 'aabbcc', 'abb', 'bcc', 'bbcc', 'aabbccdd', 'aaabb', 'cccd']
p = len(pattern)
for i in range(p):
  x.addKeyword(pattern[i])
x.setFailure()

t = len(pattern)
for i in range(t):
  target = 'aaaabbaabbccdd'
  print('YES') if x.findString(target) else print('NO')
  

YES
YES
YES
YES
YES
YES
YES
YES
YES


In [136]:
x = main()
pattern =  ['cccd']
p = len(pattern)
for i in range(p):
  x.addKeyword(pattern[i])
x.setFailure()

t = len(pattern)
for i in range(t):
  target = 'aaaabbaabbccdd'
  print('YES') if x.findString(target) else print('NO')

NO


결과 : 직접 구현한 것이 아니라 급하게 찾아서 붙여넣는 정도밖에 되지않았습니다. 결과지 역시도 문자열 전체가 입력되면 yes가 나오고 따로 잘못된 문자를 넣으면 NO가 나오면서 완전한 실패를 하였다. 문자열 매칭은 처음해보았는데 아직 제대로 이해 및 구현을 하지 못하였습니다.

이후 다시 수정하여 제출하도록 하겠습니다.